In [127]:
import numpy as np
import random
import os
import keras as k

In [3]:
import tensorflow
from keras.models import save_model, load_model, Model
from keras.layers import Input, Dropout, BatchNormalization, LeakyReLU, concatenate
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Conv2DTranspose
import matplotlib.pyplot as plt
from skimage import io
from skimage.transform import resize

In [4]:
X = np.load('CT_withtumor_input.npy')
mask = np.load('mask_withtumor_input.npy')

In [5]:
X = X[:,200:456,150:406]
mask = mask[:,200:456,150:406]

In [18]:
def improcess(X,y,patchnum):
    size = X.shape[0]
    X_train = np.empty((patchnum,256,256,3))
    y_train = np.empty((patchnum,256,256,3))
    for i in range(patchnum):
        index = random.randint(0,size-1)
        for j in range(3):
            X_train[i,:,:,j] = X[index,:,:]
            y_train[i,:,:,j] = y[index,:,:]
    
    return X_train,y_train

In [63]:
def Conv2d_BN(x, nb_filter, kernel_size, strides=(1,1), padding='same'):

    x = Conv2D(nb_filter, kernel_size, strides=strides, padding=padding)(x)

    x = BatchNormalization(axis=3)(x)

    x = LeakyReLU(alpha=0.1)(x)

    return x

 

def Conv2dT_BN(x, filters, kernel_size, strides=(2,2), padding='same'):

    x = Conv2DTranspose(filters, kernel_size, strides=strides, padding=padding)(x)

    x = BatchNormalization(axis=3)(x)

    x = LeakyReLU(alpha=0.1)(x)

    return x

 

inpt = Input(shape=(input_size_1, input_size_2, 3))

 

conv1 = Conv2d_BN(inpt, 8, (3, 3))

conv1 = Conv2d_BN(conv1, 8, (3, 3))

pool1 = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(conv1)

 

conv2 = Conv2d_BN(pool1, 16, (3, 3))

conv2 = Conv2d_BN(conv2, 16, (3, 3))

pool2 = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(conv2)

 

conv3 = Conv2d_BN(pool2, 32, (3, 3))

conv3 = Conv2d_BN(conv3, 32, (3, 3))

pool3 = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(conv3)

 

conv4 = Conv2d_BN(pool3, 64, (3, 3))

conv4 = Conv2d_BN(conv4, 64, (3, 3))

pool4 = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(conv4)

 

conv5 = Conv2d_BN(pool4, 128, (3, 3))

conv5 = Dropout(0.5)(conv5)

conv5 = Conv2d_BN(conv5, 128, (3, 3))

conv5 = Dropout(0.5)(conv5)

 

convt1 = Conv2dT_BN(conv5, 64, (3, 3))

concat1 = concatenate([conv4, convt1], axis=3)

concat1 = Dropout(0.5)(concat1)

conv6 = Conv2d_BN(concat1, 64, (3, 3))

conv6 = Conv2d_BN(conv6, 64, (3, 3))

 

convt2 = Conv2dT_BN(conv6, 32, (3, 3))

concat2 = concatenate([conv3, convt2], axis=3)

concat2 = Dropout(0.5)(concat2)

conv7 = Conv2d_BN(concat2, 32, (3, 3))

conv7 = Conv2d_BN(conv7, 32, (3, 3))

 

convt3 = Conv2dT_BN(conv7, 16, (3, 3))

concat3 = concatenate([conv2, convt3], axis=3)

concat3 = Dropout(0.5)(concat3)

conv8 = Conv2d_BN(concat3, 16, (3, 3))

conv8 = Conv2d_BN(conv8, 16, (3, 3))

 

convt4 = Conv2dT_BN(conv8, 8, (3, 3))

concat4 = concatenate([conv1, convt4], axis=3)

concat4 = Dropout(0.5)(concat4)

conv9 = Conv2d_BN(concat4, 8, (3, 3))

conv9 = Conv2d_BN(conv9, 8, (3, 3))

conv9 = Dropout(0.5)(conv9)

outpt = Conv2D(filters=3, kernel_size=(1,1), strides=(1,1), padding='same', activation='sigmoid')(conv9)

 

model = Model(inpt, outpt)

model.compile(loss='mean_squared_error', optimizer='Nadam', metrics=['accuracy'])

model.summary()

 

itr = 3000

S = []

for i in range(itr):

    print("iteration = ", i+1)

    if i < 500:

        bs = 4

    elif i < 2000:

        bs = 8

    elif i < 5000:

        bs = 16

    else:

        bs = 32

    train_X, train_Y = improcess(X,mask,bs)

    model.fit(train_X, train_Y, epochs=1, verbose=0)

    if i % 100 == 99:

        save_model(model, 'unet.h5')

        


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_115 (Conv2D)             (None, 256, 256, 8)  224         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_133 (BatchN (None, 256, 256, 8)  32          conv2d_115[0][0]                 
__________________________________________________________________________________________________
leaky_re_lu_133 (LeakyReLU)     (None, 256, 256, 8)  0           batch_normalization_133[0][0]    
__________________________________________________________________________________________________
conv2d_116

In [215]:
class Point(object):
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def getX(self):
        return self.x
    def getY(self):
        return self.y

def getGrayDiff(img,currentPoint,tmpPoint):
    result = abs(img[currentPoint.x,currentPoint.y] - img[tmpPoint.x,tmpPoint.y])
    return result

def selectConnects(p):
    if p != 0:
        connects = [Point(-1, -1), Point(0, -1), Point(1, -1), Point(1, 0), Point(1, 1), \
                    Point(0, 1), Point(-1, 1), Point(-1, 0)]
    else:
        connects = [ Point(0, -1),  Point(1, 0),Point(0, 1), Point(-1, 0)]
    return connects

def regionGrow(img,seed,thresh,p = 1):
    height, weight = img.shape
    seedMark = np.zeros(img.shape)
    seedList = []
    seedList.append(seed)
    label = 1
    connects = selectConnects(p)
    while(len(seedList)>0):
        currentPoint = seedList.pop(0)

        seedMark[currentPoint.x,currentPoint.y] = label
        for i in range(8):
            tmpX = currentPoint.x + connects[i].x
            tmpY = currentPoint.y + connects[i].y
            if (tmpX < 0):
                continue
            elif (tmpY < 0):
                continue
            elif (tmpX >=height):
                continue
            elif (tmpY >= weight):
                continue
            grayDiff = getGrayDiff(img,currentPoint,Point(tmpX,tmpY))
            if grayDiff < thresh and seedMark[tmpX,tmpY] == 0:
                seedMark[tmpX,tmpY] = label
                seedList.append(Point(tmpX,tmpY))
    return seedMark

In [260]:
def dice_coef(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = sum(y_true_f * y_pred_f)
    true_area = sum(y_true_f)
    pred_area = sum(y_pred_f)
    if true_area == 0:
        if pred_area == 0:
            dice = 1
        else:
            dice = 1-pred_area/(256*256)
    else:
        dice = 2*intersection/(sum(y_true_f)+sum(y_pred_f))
    return dice


In [254]:
def img_predict(X_test,y_test,model):
    img_test = np.empty([1,256,256,3])
    for i in range(3):
        img_test[0,:,:,i] = X_test
    pred_Y = model.predict(img_test)
    y_pred = np.empty((256,256))
    for i in range(256):
        for j in range(256):
            y_pred[i,j] = np.mean(pred_Y[0,i,j])
    if y_pred.max()<0.4:
        binaryImg1 = np.zeros([256,256])
        binaryImg2 = np.zeros([256,256])
    else:
        binaryImg1 = np.zeros([256,256])
        index1 = np.where(y_pred > 0.5)
        binaryImg1[index1] = 1
        index2 = np.where(y_pred == y_pred.max())
        if(index2[0].shape == 1):
            seed = Point(index2[0],index2[1])
        else:
            seed = Point(index2[0][0],index2[1][0])
        binaryImg2 = regionGrow(X_test,seed,0.0067)
    dice1 = dice_coef(binaryImg1,y_test)
    dice2 = dice_coef(binaryImg2,y_test)
    return binaryImg1,binaryImg2,dice1,dice2

In [64]:
save_model(model, 'unet.h5')

In [114]:
model = load_model('unet.h5')

In [ ]:
X_test = np.load("E:/Teddy/npys/CT_test.npy")
y_test = np.load("E:/Teddy/npys/mask_test.npy")

test_X, test_Y = X_test[:,200:456,150:406],y_test[:,200:456,150:406]

In [262]:
test_size = test_X.shape[0]
dice1 = np.empty([test_size])
dice2 = np.empty([test_size])
y_pred1 = np.empty([test_size,256,256])
y_pred2 = np.empty([test_size,256,256])
for i in range(test_size):
    y_pred1[i,:,:],y_pred2[i,:,:],dice1[i],dice2[i] = img_predict(test_X[i], test_Y[i],model)
#dice1 = dice1/test_size
#dice2 = dice2/test_size

In [284]:
dice_coef1 = sum(dice1)/test_size
dice_coef2 = sum(dice2)/test_size

In [285]:
print(dice_coef1,dice_coef2)

0.6820058357734182 0.5818199512165754
